#패키지 불러오기

In [53]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
% matplotlib inline
from datetime import datetime
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
from sklearn.utils import class_weight
from sklearn.preprocessing import RobustScaler
from sklearn.neighbors import KNeighborsClassifier

#데이터 불러오기 및 전처리

In [4]:
data_train = pd.read_csv('/content/train.csv')

In [5]:
data_train = data_train.fillna('unknown')

X = data_train.drop(columns=['index', 'FLAG_MOBIL', 'child_num', 'credit'])
y = data_train['credit']

X_encoded = pd.get_dummies(X, columns=['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type'])

In [6]:
X_encoded

,income_total,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,family_size,begin_month,gender_F,gender_M,car_N,car_Y,reality_N,reality_Y,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,occyp_type_unknown
0,202500.0,-13899,-4709,0,0,0,2.0,-6.0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,247500.0,-11380,-1540,0,0,1,3.0,-5.0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,450000.0,-19087,-4434,0,1,0,2.0,-22.0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,202500.0,-15088,-2092,0,1,0,2.0,-37.0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,157500.0,-15037,-2105,0,0,0,2.0,-26.0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,225000.0,-12079,-1984,0,0,0,4.0,-2.0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26453,180000.0,-15291,-2475,0,0,0,2.0,-47.0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
26454,292500.0,-10082,-2015,0,0,0,2.0,-25.0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26455,171000.0,-10145,-107,0,0,0,1.0,-59.0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [7]:
num_features = [(0, 'income_total'), (1, 'DAYS_BIRTH'), (2, 'DAYS_EMPLOYED'), (6, 'family_size'), (7, 'begin_month')]
already = [0, 1, 2, 6, 7]
features = list(range(54))

for i in already:
  features.remove(i)

scaler = RobustScaler()
X_dummy = X_encoded.iloc[:, features].reset_index(drop=True)
X_robust = pd.DataFrame(scaler.fit_transform(X_encoded.iloc[:, [t[0] for t in num_features]]), columns=[t[1] for t in num_features])
X_final = pd.concat([X_robust,X_dummy ], axis=1)

In [8]:
X_final

,income_total,DAYS_BIRTH,DAYS_EMPLOYED,family_size,begin_month,work_phone,phone,email,gender_F,gender_M,car_N,car_Y,reality_N,reality_Y,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,occyp_type_unknown
0,0.434783,0.235934,-1.154406,0.0,0.666667,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0.869565,0.596564,-0.000364,1.0,0.703704,0,0,1,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,2.826087,-0.506800,-1.054261,0.0,0.074074,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0.434783,0.065712,-0.201384,0.0,-0.481481,0,1,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0.000000,0.073014,-0.206118,0.0,-0.074074,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,0.652174,0.496492,-0.162054,2.0,0.814815,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26453,0.217391,0.036650,-0.340859,0.0,-0.851852,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
26454,1.304348,0.782391,-0.173343,0.0,-0.037037,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26455,0.130435,0.773372,0.521486,-1.0,-1.296296,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [10]:
set = [X_train, X_valid, X_test, y_train, y_valid, y_test]
for i in set:
  print(i.shape)

(15873, 54)
(5292, 54)
(5292, 54)
(15873,)
(5292,)
(5292,)


In [11]:
X_train

,income_total,DAYS_BIRTH,DAYS_EMPLOYED,family_size,begin_month,work_phone,phone,email,gender_F,gender_M,car_N,car_Y,reality_N,reality_Y,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,occyp_type_unknown
6654,-0.217391,0.830064,0.329206,0.0,-0.703704,1,1,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
12015,0.217391,0.659556,-0.343773,1.0,0.074074,1,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
16533,1.086957,0.689048,-0.353969,0.0,0.814815,0,1,0,0,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
12117,2.173913,0.730279,-0.320830,0.0,0.777778,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
229,-0.652174,-0.829349,133.569556,0.0,-1.000000,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4261,-0.434783,0.600429,-0.282957,0.0,0.407407,0,0,1,1,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6651,0.000000,0.592842,-0.560452,-1.0,-0.888889,1,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
11264,-0.217391,0.661847,0.313183,2.0,0.703704,1,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
23249,0.434783,-0.291768,-0.538237,0.0,-0.370370,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


test set 전처리

In [22]:
data_test = pd.read_csv('/content/test.csv')

In [39]:
data_test = data_test.fillna('unknown')

X_test = data_test.drop(columns=['index', 'FLAG_MOBIL', 'child_num'])

X_test_encoded = pd.get_dummies(X_test, columns=['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type'])

In [40]:
X_test_encoded

,income_total,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,family_size,begin_month,gender_F,gender_M,car_N,car_Y,reality_N,reality_Y,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,occyp_type_unknown
0,112500.0,-21990,365243,0,1,0,2.0,-60.0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,135000.0,-18964,-8671,0,1,0,2.0,-36.0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,69372.0,-15887,-217,1,1,0,2.0,-40.0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,112500.0,-19270,-2531,1,0,0,2.0,-41.0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,225000.0,-17822,-9385,1,0,0,2.0,-8.0,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,202500.0,-18593,-5434,1,1,0,2.0,-19.0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9996,202500.0,-10886,-1315,1,0,0,2.0,-34.0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9997,292500.0,-21016,-14018,0,0,0,2.0,-55.0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
9998,180000.0,-16541,-1085,0,1,0,2.0,-33.0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [41]:
num_features = [(0, 'income_total'), (1, 'DAYS_BIRTH'), (2, 'DAYS_EMPLOYED'), (6, 'family_size'), (7, 'begin_month')]
already = [0, 1, 2, 6, 7]
features = list(range(54))

for i in already:
  features.remove(i)

scaler = RobustScaler()
X_test_dummy = X_test_encoded.iloc[:, features].reset_index(drop=True)
X_test_robust = pd.DataFrame(scaler.fit_transform(X_test_encoded.iloc[:, [t[0] for t in num_features]]), columns=[t[1] for t in num_features])
X_test_final = pd.concat([X_test_robust,X_test_dummy ], axis=1)

# classification

naive bayes = 음수 조정 필요

ADABoost

In [47]:
from sklearn.ensemble import AdaBoostClassifier

In [48]:
clf = AdaBoostClassifier(n_estimators=30, learning_rate=1.0, random_state=2021)
clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=30, random_state=2021)

In [49]:
y_pred_adab = clf.predict_proba(X_valid)

In [55]:
log_loss(y_true=y_valid, y_pred=y_pred_adab)

1.0710560291160214

In [42]:
curr_time = datetime.now().strftime('%y%m%d%H%M%S')
X_raw = pd.read_csv('/content/test.csv', encoding='utf-8')
l_idx = X_raw['index']
submission = pd.DataFrame(np.insert(clf.predict_proba(X_test_final), 0, l_idx, axis=1), columns=['index', 0, 1, 2])
submission['index'] = submission['index'].astype('int')

In [43]:
submission.to_csv(f'submission_{curr_time}.csv', index=False)

In [64]:
y_pred_adab = clf.predict(X_valid)

In [65]:
accuracy_score(y_true=y_valid, y_pred=y_pred_adab)

0.6923658352229781

SVM

In [16]:
%time

svm_clf = SVC(kernel='rbf',
              probability = True,
              random_state=2021)
svm_clf.fit(X_train, y_train)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 7.87 µs


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=2021, shrinking=True, tol=0.001,
    verbose=False)

In [17]:
y_pred_svm = svm_clf.predict_proba(X_valid)

In [18]:
y_pred_svm

array([[0.12368415, 0.2365686 , 0.63974725],
       [0.12814615, 0.24162609, 0.63022776],
       [0.12799871, 0.24899151, 0.62300978],
       ...,
       [0.12651996, 0.23861369, 0.63486634],
       [0.11809566, 0.2414715 , 0.64043283],
       [0.12878083, 0.24026699, 0.63095218]])

In [19]:
log_loss(y_true=y_valid, y_pred=y_pred_svm)

0.8757679748936101

In [20]:
y_pred_svm = svm_clf.predict_proba(X_test)

In [21]:
log_loss(y_true=y_test, y_pred=y_pred_svm)

0.8818593835659214

In [44]:
curr_time = datetime.now().strftime('%y%m%d%H%M%S')
X_raw = pd.read_csv('/content/test.csv', encoding='utf-8')
l_idx = X_raw['index']
submission = pd.DataFrame(np.insert(svm_clf.predict_proba(X_test_final), 0, l_idx, axis=1), columns=['index', 0, 1, 2])
submission['index'] = submission['index'].astype('int')
submission.to_csv(f'submission_{curr_time}.csv', index=False)

In [67]:
y_pred_svm = svm_clf.predict(X_valid)

In [68]:
accuracy_score(y_true=y_valid, y_pred=y_pred_svm)

0.6432350718065004

XGB

In [56]:
from xgboost import plot_importance
from xgboost import XGBClassifier

In [58]:
clf_xgb = XGBClassifier(n_estimators=100, learning_rate = 0.1, max_depth = 10)

In [59]:
clf_xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [61]:
y_pred_xgb = clf_xgb.predict_proba(X_valid)
log_loss(y_true=y_valid, y_pred=y_pred_xgb)

0.758077860912287

In [62]:
curr_time = datetime.now().strftime('%y%m%d%H%M%S')
X_raw = pd.read_csv('/content/test.csv', encoding='utf-8')
l_idx = X_raw['index']
submission = pd.DataFrame(np.insert(clf_xgb.predict_proba(X_test_final), 0, l_idx, axis=1), columns=['index', 0, 1, 2])
submission['index'] = submission['index'].astype('int')
submission.to_csv(f'submission_{curr_time}.csv', index=False)

In [69]:
y_pred_xgb = clf_xgb.predict(X_valid)
accuracy_score(y_true=y_valid, y_pred=y_pred_xgb)

0.7078609221466364